In [1]:
spatialSample="p16" #p16 or spider. Key.

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd
import tangram as tg

#This takes a few minutes.

In [3]:
adata_st=sc.read_h5ad(f"/home/hezhou/_Projects/25.02.27_Jielin_Senescence/25.04.07_prepareData/_Data/dataSpatial_{spatialSample}.h5ad")
adata_sc=sc.read_h5ad("/home/hezhou/_Projects/25.02.27_Jielin_Senescence/25.04.07_prepareData/_Data/dataSnRNASeq.h5ad")

#This takes a moment.

In [4]:
adata_st.X.shape #Cell by gene.

(1808, 19465)

In [5]:
adata_sc.X.shape #Cell by gene.

(44573, 32285)

In [6]:
# sc.tl.rank_genes_groups(adata_sc,groupby="celltype",use_raw=False)

#WARNING: It seems you use rank_genes_groups on the raw count data. Please logarithmize your data before calling rank_genes_groups.

In [7]:
adata_sc2=adata_sc.copy() #Create unlinked copy.

In [8]:
sc.pp.log1p(adata_sc2)

In [9]:
# print(adata_sc2.X)

In [10]:
sc.tl.rank_genes_groups(adata_sc2,groupby="celltype",use_raw=False)

#This takes a minute or so.

In [11]:
markers_df=pd.DataFrame(adata_sc2.uns["rank_genes_groups"]["names"]).iloc[0:100,:] #Take top 100 marker genes per cell type.

In [12]:
markers_df

,Cardiomyocytes,Dentritic cells,Endothelial cells,Epicardial cells,Fibroblasts,Immune-B cells,Immune-T cells,Macrophages,Pericytes,Smooth muscle cells
0,Ttn,Dock10,Ptprb,Slit3,Col8a1,Bank1,Skap1,Dock2,Dlc1,Kcnab1
1,Ryr2,Tbc1d8,Cdh13,Efna5,Gpc6,Arhgap15,Dock2,Fyb,Prkg1,Prkg1
2,Sorbs1,Dock2,Flt1,Ncam1,Rbms3,Ripor2,Ptprc,Zeb2,Notch3,Ctnna3
3,Tnnt2,Wdfy4,Pecam1,Zfpm2,Tenm3,Aff3,Gm2682,Ptprc,Pdgfrb,Myh11
4,Rbm20,Flt3,Ptprm,Wt1,Col5a2,Bach2,Arhgap15,Myo1f,Gpc6,Dmd
...,...,...,...,...,...,...,...,...,...,...
95,Myl2,Ap1s3,Nova2,Magi2,Adamts6,B3gnt5,Gm26740,St18,Meg3,Tbx2
96,Cdh13,Gm15987,Dgkh,mt-Co2,Tmem45a,Stap1,Apbb1ip,Cyth4,Epas1,Cyp7b1
97,Grm1,Prex1,Dock6,Gm12840,Ext1,Ly86,Pdcd4,Gng2,Adap2,Gucy1b1
98,Gm35330,Birc3,Nos3,B830012L14Rik,Svep1,Sema4d,Cd96,Tnfrsf11a,Serpine2,Rgs6


In [13]:
markers=list(np.unique(markers_df.melt().value.values))

In [14]:
len(markers) #791 marker genes.

791

In [15]:
tg.pp_adatas(adata_sc,adata_st,genes=markers) #741 training genes.

INFO:root:741 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:14660 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [16]:
ad_map=tg.map_cells_to_space(adata_sc,adata_st,mode="cells") #https://github.com/broadinstitute/Tangram/blob/master/tangram/mapping_utils.py

#This takes about an hour.

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 741 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.351, KL reg: 0.129
Score: 0.668, KL reg: 0.001
Score: 0.687, KL reg: 0.001
Score: 0.689, KL reg: 0.001
Score: 0.690, KL reg: 0.001
Score: 0.690, KL reg: 0.001
Score: 0.691, KL reg: 0.001
Score: 0.691, KL reg: 0.001
Score: 0.691, KL reg: 0.001
Score: 0.691, KL reg: 0.001


INFO:root:Saving results..


In [17]:
ad_map

AnnData object with n_obs × n_vars = 44573 × 1808
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'sample', 'celltype'
    var: 'nCount_Spatial', 'nFeature_Spatial', 'uniform_density', 'rna_count_based_density'
    uns: 'train_genes_df', 'training_history'

In [18]:
ad_map.X

array([[6.9648188e-08, 7.7271494e-08, 1.9294797e-07, ..., 1.6802518e-07,
        9.8259228e-08, 1.8212037e-07],
       [4.5804708e-07, 1.8967145e-07, 7.6436031e-08, ..., 2.4808224e-07,
        1.3044571e-07, 4.4963226e-08],
       [6.7758810e-06, 4.9859068e-06, 4.3690757e-06, ..., 3.8332209e-06,
        3.6674967e-06, 6.5631893e-06],
       ...,
       [3.6604635e-07, 6.0098989e-07, 1.9199965e-07, ..., 8.4058570e-07,
        2.5879372e-06, 1.3976539e-06],
       [5.0898740e-07, 2.6990460e-07, 3.6171068e-07, ..., 3.4411539e-07,
        6.2612844e-07, 3.4359761e-07],
       [6.3302593e-08, 6.5401530e-08, 1.8500532e-07, ..., 1.6381624e-07,
        1.8400192e-08, 7.3830016e-08]], dtype=float32)

In [19]:
tg.project_cell_annotations(ad_map,adata_st,annotation="celltype")

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


In [20]:
adata_st.obsm

AxisArrays with keys: tangram_ct_pred

In [21]:
adata_st.obsm["tangram_ct_pred"]

,Fibroblasts,Endothelial cells,Immune-T cells,Macrophages,Cardiomyocytes,Smooth muscle cells,Immune-B cells,Pericytes,Dentritic cells,Epicardial cells
AACACGTTGATACCGC-1,9.619183,4.209319,0.000574,5.481513,1.799357,0.994209,0.122888,1.849255,0.133543,0.302334
AACAGGCCATTGTCAC-1,8.687117,7.135786,0.000425,4.224581,1.625298,0.421930,0.734737,0.550454,0.127161,1.055588
AACAGTATCCGCCTGG-1,13.345734,15.248071,0.138977,11.379629,2.532132,0.659650,0.126224,2.343225,1.931582,0.088741
AACATACTCCACAGTT-1,10.670270,11.436491,0.000540,6.080512,2.687811,0.000179,0.000472,3.408780,0.149829,1.851625
AACATAGGTTGGCACC-1,4.944032,9.074845,0.022946,2.747240,1.042944,0.000254,0.039295,0.665139,0.088044,2.256658
...,...,...,...,...,...,...,...,...,...,...
TGTTCGTGGCGTCGTG-1,5.995681,4.642933,0.486700,4.879591,1.257069,0.000173,0.169890,3.121660,0.000187,1.015401
TGTTGCCAGTCGCCTG-1,2.506289,5.144205,0.201334,5.701785,1.374159,0.000336,0.117662,0.730019,0.103629,0.068572
TGTTGGCCTGTAGCGG-1,22.806515,2.919701,0.177095,3.430548,0.108269,0.156520,0.000382,0.000829,0.000141,0.002196
TGTTGGTGCGGAATCA-1,11.223494,5.873017,0.271154,1.595921,1.055309,0.089263,0.158152,1.140372,0.000362,0.000414


In [22]:
np.sum(adata_st.obsm["tangram_ct_pred"],axis=1) #Row sums.

AACACGTTGATACCGC-1    24.512173
AACAGGCCATTGTCAC-1    24.563078
AACAGTATCCGCCTGG-1    47.793964
AACATACTCCACAGTT-1    36.286509
AACATAGGTTGGCACC-1    20.881396
                        ...    
TGTTCGTGGCGTCGTG-1    21.569286
TGTTGCCAGTCGCCTG-1    15.947990
TGTTGGCCTGTAGCGG-1    29.602197
TGTTGGTGCGGAATCA-1    21.407458
TGTTGGTGGACTCAGG-1    18.479068
Length: 1808, dtype: float64

In [23]:
np.sum(adata_st.obsm["tangram_ct_pred"]) #Column sums.

Fibroblasts            18689.002637
Endothelial cells       8759.000692
Immune-T cells           429.999996
Macrophages            10181.001077
Cardiomyocytes          1931.000010
Smooth muscle cells      355.000003
Immune-B cells           282.000004
Pericytes               2703.000104
Dentritic cells          263.000005
Epicardial cells         980.000039
dtype: float64

In [24]:
np.sum(np.sum(adata_st.obsm["tangram_ct_pred"])) #Total sum. 44573.

np.float64(44573.00456662309)

Tangram plotting function doesn't work.

In [25]:
adata_st.obsm["tangram_ct_pred"].to_csv(f"/home/hezhou/_Projects/25.02.27_Jielin_Senescence/25.04.07_spatialCellTypes/1.4_resultTangram/{spatialSample}.csv")